<a href="https://colab.research.google.com/github/mraaheb/DataMining/blob/main/Phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1 — Load the Preprocessed Dataset

We begin Phase 3 by loading the preprocessed dataset generated in Phase 2.  
This dataset is already cleaned and encoded, so it is ready for applying K-means clustering.


In [7]:
import pandas as pd

# Load the preprocessed dataset from GitHub

df = pd.read_csv("lung_cancer_preprocessed.csv", index=False)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

df.head()


TypeError: read_csv() got an unexpected keyword argument 'index'